# Introduction to Anomaly Detection

Reference: https://www.kaggle.com/code/ysjang0926/kor-introduction-to-anomaly-detection-r01#Semi-Supervised-Classification-using-AutoEncoders

## Prerequisites

- 필요한 라이브러리 다운로드

In [ ]:
%pip install kagglehub
%pip install pandas
%pip install numpy==1.26
%pip install matplotlib
%pip install scikit-learn
%pip install seaborn
%pip install --upgrade keras
%pip install tensorflow
%pip install setuptools

## Dataset Preparation

- 필요한 데이터셋 다운로드
- Pandas 데이터프레임 로드

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("mlg-ulb/creditcardfraud")

print("Path to dataset files:", path)

In [ ]:
import pandas as pd 
import numpy as np
import seaborn as sns
sns.set_theme(style="whitegrid")
np.random.seed(203)

data = pd.read_csv(f"{path}/creditcard.csv")
data.head()

In [ ]:
data["Time"] = data["Time"].apply(lambda x : x / 3600 % 24)
data.head()

- Time: 첫 번째 Transaction으로부터 경과된 시간(초)
- V1 ~ V28: PCA 변환된 변수
- Amount: 거래 금액
- Class: Target 변수 (0: 정상, 1: 비정상)

In [ ]:
import matplotlib.pyplot as plt

colors = ["#0101DF", "#DF0101"]
LABELS = ["Non-Fraud", "Fraud"]
sns.countplot(x='Class', data=data, palette=colors)
plt.title('Class Distribution \n (0 : Non-Fraud / 1 : Fraud)', fontsize=13)
plt.xticks(range(2), LABELS)
plt.xlabel("Class")
plt.ylabel("Frequency")

In [ ]:
vc = data['Class'].value_counts().to_frame().reset_index()
vc['percent'] = vc["Class"].apply(lambda x : round(100*float(x) / len(data), 2))
vc = vc.rename(columns = {"index" : "Target", "Class" : "Count"})
vc

In [ ]:
# 각 컬럼의 기초 통계값 확인
data.describe()

In [ ]:
# null 값 체크: 결측치 갯수
data.isnull().sum().max()


In [ ]:
# null 값 체크: True/False
data.isnull().values.any()

- 사기 거래가 전체의 0.17%이어서 불균형이 크기 때문에, Non-fraud 거래 내역의 1,000행만 사용

In [ ]:
non_fraud = data[data['Class'] == 0].sample(1000)
fraud = data[data['Class'] == 1]

df = pd.concat([non_fraud, fraud]).sample(frac=1).reset_index(drop=True)
X = df.drop(['Class'], axis = 1).values
Y = df["Class"].values

## Visualize Fraud and Non-Fraud Transactions

- 이상치가 있다는 것은 기존의 데이터가 어느 정도 패턴을 갖고있다는 것을 의미
- `t-SNE`를 사용하여 Fraud와 Non-Fraud 거래의 특성을 시각화

### t-SNE(t-Distributed Stochastic Neighbor Embedding)란?

- 높은 차원의 복잡한 데이터를 2차원에 차원 축소하는 방법
- 각 데이터 포인트 주변으로 유사도를 계산하여, 각 데이터 포인트를 2차원에서 원본 특성 공간에서 비슷한 데이터 구조는 가깝게 배치하고, 구조가 다른 데이터는 멀리 배치

#### PCA와 차이점

- PCA는 Linear한 방법을 사용하지만, t-SNE는 Non-linear한 방법을 사용
- t-SNE는 Input Feature를 확인하기 어렵기 떄문에 t-SNE 결과만 가지고 무언가를 추론하기 어려움

In [ ]:
from sklearn.manifold import TSNE

def tsne_plot(x1, y1, name="graph.png"):
    tsne = TSNE(n_components=2, random_state=0)
    X_t = tsne.fit_transform(x1)

    plt.figure(figsize=(12, 8))
    plt.scatter(X_t[np.where(y1 == 0), 0], X_t[np.where(y1 == 0), 1], marker='o', color='g', alpha=0.8, label='Non Fraud')
    plt.scatter(X_t[np.where(y1 == 1), 0], X_t[np.where(y1 == 1), 1], marker='o', color='r', alpha=0.8, label='Fraud')

    plt.legend(loc='best')
    plt.show()

In [ ]:
tsne_plot(X, Y, "original.png")

- Fraud 거래에 가까운 Non-Fraud 거래가 많기 때문에 정확하게 분류하기 어렵다는 것을 알 수 있음

## AutoEncoders to the Rescue

### AutoEncoder란?

- 출력이 입력과 동일한 특수한 유형의 Neural Network 아키텍처
- 입력 데이터를 압축한(Encoder) 후 다시 복원하는(Decoder) 방법으로 학습
- 고차원 상의 입력 데이터를 저차원 공간으로 Mapping하여 Latent Representation으로 표현하였다가, 다시 이렵과 같은 고차원의 공간으로 복원
- 데이터가 잘 복원된 경우 저차원의 데이터 특성 공간을 잘 파악한 것임

### 가정

- 정상 관측치들은 불량 관측치보다 더 잘 복원될 것임
- Reconstruct의 오차 이상치 점수를 산출하여, 특정 임계값보다 큰 경우 불량 관측치로 판단

In [ ]:
from keras.layers import Input, Dense
from keras import regularizers

## input layer 
input_layer = Input(shape=(X.shape[1],))

## encoding part
encoded = Dense(100, activation='tanh', activity_regularizer=regularizers.l1(10e-5))(input_layer)
encoded = Dense(50, activation='relu')(encoded)

## decoding part
decoded = Dense(50, activation='tanh')(encoded)
decoded = Dense(100, activation='tanh')(decoded)

## output layer
output_layer = Dense(X.shape[1], activation='relu')(decoded)

In [ ]:
from keras import Model

autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer="adadelta", loss="mse")

- 각 변수의 범위가 다르기 때문에 MinMaxScaler를 사용하여 정규화

In [ ]:
from sklearn import preprocessing

x = data.drop(["Class"], axis=1)
y = data["Class"].values

x_scale = preprocessing.MinMaxScaler().fit_transform(x.values)
x_norm, x_fraud = x_scale[y == 0], x_scale[y == 1]

In [ ]:
autoencoder.fit(
    x_norm[0:2000],
    x_norm[0:2000],
    batch_size=256,
    epochs=10,
    shuffle=True,
    validation_split=0.20,
)

In [ ]:
from keras import Sequential

hidden_representation = Sequential()
hidden_representation.add(autoencoder.layers[0])
hidden_representation.add(autoencoder.layers[1])
hidden_representation.add(autoencoder.layers[2])

In [ ]:
norm_hid_rep = hidden_representation.predict(x_norm[:3000])
fraud_hid_rep = hidden_representation.predict(x_fraud)

In [ ]:
rep_x = np.append(norm_hid_rep, fraud_hid_rep, axis=0)
y_n = np.zeros(norm_hid_rep.shape[0])
y_f = np.ones(fraud_hid_rep.shape[0])
rep_y = np.append(y_n, y_f)
tsne_plot(rep_x, rep_y, "latent_representation.png")

- Fraud 거래와 Non-Frad 거래가 잘 구분되는 것을 확인할 수 있음
- Linear하게 구분할 수 있음

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score

train_x, val_x, train_y, val_y = train_test_split(rep_x, rep_y, test_size=0.25)
clf = LogisticRegression(solver="lbfgs").fit(train_x, train_y)
pred_y = clf.predict(val_x)

print("")
print("Classification Report: ")
print(classification_report(val_y, pred_y))

print("")
print(
    "Logistic Regression Accuracy Score: ",
    str(round(accuracy_score(val_y, pred_y) * 100, 3)) + "%",
)

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_pred=pred_y, y_true=val_y)
cmp = ConfusionMatrixDisplay(cm)
cmp.plot(cmap=plt.cm.Blues)